**Artificial Neural Networks**

In [1]:
import sys

assert sys.version_info >= (3, 7)

In [2]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

In [ ]:
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [ ]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "ann"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Des neurones biologiques aux neurones artificiels
# Le perceptron

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris(as_frame=True)
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = (iris.target == 0)  # Iris setosa

per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

X_new = [[2, 0.5], [3, 1]]
y_pred = per_clf.predict(X_new)  # predicts True and False for these 2 flowers

In [ ]:
y_pred

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(loss="perceptron", penalty=None,
                        learning_rate="constant", eta0=1, random_state=42)
sgd_clf.fit(X, y)
assert (sgd_clf.coef_ == per_clf.coef_).all()
assert (sgd_clf.intercept_ == per_clf.intercept_).all()

 L'algorithme d'apprentissage du perceptron ressemble fortement à la descente de gradient stochastique. En fait, la classe Perceptron de Scikit-Learn est équivalente à l'utilisation d'un SGDClassifier avec les hyperparamètres suivants : loss="perceptron", learning_rate="constant", eta0=1 (le taux d'apprentissage) et penalty=None (pas de régularisation).

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

a = -per_clf.coef_[0, 0] / per_clf.coef_[0, 1]
b = -per_clf.intercept_ / per_clf.coef_[0, 1]
axes = [0, 5, 0, 2]
x0, x1 = np.meshgrid(
    np.linspace(axes[0], axes[1], 500).reshape(-1, 1),
    np.linspace(axes[2], axes[3], 200).reshape(-1, 1),
)
X_new = np.c_[x0.ravel(), x1.ravel()]
y_predict = per_clf.predict(X_new)
zz = y_predict.reshape(x0.shape)
custom_cmap = ListedColormap(['#9898ff', '#fafab0'])

plt.figure(figsize=(7, 3))
plt.plot(X[y == 0, 0], X[y == 0, 1], "bs", label="Not Iris setosa")
plt.plot(X[y == 1, 0], X[y == 1, 1], "yo", label="Iris setosa")
plt.plot([axes[0], axes[1]], [a * axes[0] + b, a * axes[1] + b], "k-",
         linewidth=3)
plt.contourf(x0, x1, zz, cmap=custom_cmap)
plt.xlabel("Petal length")
plt.ylabel("Petal width")
plt.legend(loc="lower right")
plt.axis(axes)
plt.show()

**Les fonctions d'activation**

In [ ]:
from scipy.special import expit as sigmoid

def relu(z):
    return np.maximum(0, z)

def derivative(f, z, eps=0.000001):
    return (f(z + eps) - f(z - eps))/(2 * eps)

max_z = 4.5
z = np.linspace(-max_z, max_z, 200)

plt.figure(figsize=(11, 3.1))

plt.subplot(121)
plt.plot([-max_z, 0], [0, 0], "r-", linewidth=2, label="Heaviside")
plt.plot(z, relu(z), "m-.", linewidth=2, label="ReLU")
plt.plot([0, 0], [0, 1], "r-", linewidth=0.5)
plt.plot([0, max_z], [1, 1], "r-", linewidth=2)
plt.plot(z, sigmoid(z), "g--", linewidth=2, label="Sigmoid")
plt.plot(z, np.tanh(z), "b-", linewidth=1, label="Tanh")
plt.grid(True)
plt.title("Activation functions")
plt.axis([-max_z, max_z, -1.65, 2.4])
plt.gca().set_yticks([-1, 0, 1, 2])
plt.legend(loc="lower right", fontsize=13)

plt.subplot(122)
plt.plot(z, derivative(np.sign, z), "r-", linewidth=2, label="Heaviside")
plt.plot(0, 0, "ro", markersize=5)
plt.plot(0, 0, "rx", markersize=10)
plt.plot(z, derivative(sigmoid, z), "g--", linewidth=2, label="Sigmoid")
plt.plot(z, derivative(np.tanh, z), "b-", linewidth=1, label="Tanh")
plt.plot([-max_z, 0], [0, 0], "m-.", linewidth=2)
plt.plot([0, max_z], [1, 1], "m-.", linewidth=2)
plt.plot([0, 0], [0, 1], "m-.", linewidth=1.2)
plt.plot(0, 1, "mo", markersize=5)
plt.plot(0, 1, "mx", markersize=10)
plt.grid(True)
plt.title("Derivatives")
plt.axis([-max_z, max_z, -0.2, 1.2])

save_fig("activation_functions_plot")
plt.show()

Voici une version simplifiée des explications des fonctions d'activation couramment utilisées dans les réseaux de neurones : Heaviside, Sigmoid, ReLU et Tanh :

### 1. Fonction d'Échelon de Heaviside
Cette fonction donne 0 si l'entrée est négative et 1 si elle est positive. Elle est simple mais ne permet pas une mise à jour efficace lors de l'apprentissage car elle n'est pas différentiable.

### 2. Fonction Sigmoïde
La sigmoïde transforme les entrées en valeurs entre 0 et 1, interprétables comme des probabilités. Elle est idéale pour les tâches de classification binaire. Toutefois, elle peut ralentir l'apprentissage dans les réseaux profonds à cause du problème de "gradient qui disparaît".

### 3. Unité Linéaire Rectifiée (ReLU)
ReLU est très populaire, surtout pour les réseaux profonds, car elle transmet directement les valeurs positives et annule les négatives, ce qui accélère l'apprentissage. Cependant, elle peut causer des problèmes lorsque toutes les sorties deviennent nulles.

### 4. Tangente Hyperbolique (Tanh)
Tanh est similaire à la sigmoïde mais varie de -1 à 1, ce qui aide à centrer les données et peut conduire à un apprentissage plus stable et rapide. Comme la sigmoïde, elle peut aussi être affectée par le problème de gradient qui disparaît.

Chaque fonction a ses avantages et est choisie selon le besoin spécifique de l'application et du type de réseau de neurones utilisé.

## Regression MLPs

les MLP (réseaux de neurones multicouches) peuvent être utilisés pour des tâches de régression. Si vous souhaitez prédire une valeur unique (par exemple, le prix d'une maison, compte tenu de nombreuses de ses caractéristiques), vous avez juste besoin d'un seul neurone de sortie : sa sortie est la valeur prédite. Pour la régression multivariée (c'est-à-dire pour prédire plusieurs valeurs à la fois), vous avez besoin d'un neurone de sortie par dimension de sortie.

Scikit-Learn comprend une classe MLPRegressor. Utilisons-la pour construire un MLP avec trois couches cachées composées de 50 neurones chacune, et entraînons-le sur le jeu de données du logement en Californie.

Pour simplifier, nous utiliserons la fonction fetch_california_housing() de Scikit-Learn pour charger les données. Ce jeu de données est plus simple que celui que nous avons utilisé précédemment.

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

mlp_reg = MLPRegressor(hidden_layer_sizes=[50, 50, 50], random_state=42)
pipeline = make_pipeline(StandardScaler(), mlp_reg)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_valid)
rmse = mean_squared_error(y_valid, y_pred, squared=False)
rmse

Notez que ce MLP n'utilise aucune fonction d'activation pour la couche de sortie, donc il est libre de produire toute valeur qu'il souhaite. Cela est généralement acceptable, mais si vous voulez garantir que la sortie sera toujours positive, vous devriez alors utiliser la fonction d'activation ReLU dans la couche de sortie, ou la fonction d'activation softplus, qui est une variante lisse de ReLU : softplus(z) = log(1 + exp(z)). Softplus est proche de 0 lorsque z est négatif, et proche de z lorsque z est positif. Enfin, si vous voulez garantir que les prédictions seront toujours dans une plage de valeurs donnée, alors vous devriez utiliser la fonction sigmoid ou la tangente hyperbolique, et mettre à l'échelle les cibles vers la plage appropriée : de 0 à 1 pour le sigmoid et de –1 à 1 pour tanh. Malheureusement, la classe MLPRegressor ne prend pas en charge les fonctions d'activation dans la couche de sortie.

# Implémentation des MLP avec Keras
## Construire un classificateur d'images en utilisant l'API séquentielle
### Utilisation de Keras pour charger le jeu de données

Pour commencer à travailler avec le dataset Fashion MNIST, vous pouvez utiliser les fonctions fournies par Keras pour charger facilement ce dataset. Voici comment charger le dataset Fashion MNIST et le diviser en ensembles d'entraînement, de validation et de test :

In [ ]:
import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

L'ensemble d'apprentissage contient 60 000 images en niveaux de gris, de 28x28 pixels chacune :

In [ ]:
X_train.shape

L'intensité de chaque pixel est représentée par un octet (0 à 255) :

In [ ]:
X_train.dtype

Mettons à l'échelle les intensités des pixels dans la plage 0-1 et convertissons-les en nombres flottants en les divisant par 255 :

In [ ]:
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.

Vous pouvez tracer une image en utilisant la fonction `imshow()` de Matplotlib, avec une carte de couleurs `'binaire'' :

In [ ]:
plt.imshow(X_train[0], cmap="binary")
plt.axis('off')
plt.show()

Les étiquettes sont les identifiants des classes (représentés sous forme de uint8), de 0 à 9 :

In [ ]:
y_train

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

So the first image in the training set is an ankle boot:

In [ ]:
class_names[y_train[0]]

Jetons un coup d'œil sur un échantillon des images de l'ensemble de données :

In [ ]:
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]])
plt.subplots_adjust(wspace=0.2, hspace=0.5)

save_fig("fashion_mnist_plot")
plt.show()

### Création du modèle à l'aide de l'API séquentielle

In [ ]:
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=[28, 28]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

Voici une version simplifiée de la description du code :

- **Initialisation du germe aléatoire :** On commence par fixer le germe aléatoire de TensorFlow pour que les résultats soient les mêmes à chaque exécution. Cela garantit que les poids aléatoires dans le réseau sont constants d'une exécution à l'autre.
  
- **Création du modèle Séquentiel :** On utilise le modèle le plus simple de Keras, appelé Séquentiel, qui consiste en une pile de couches connectées les unes après les autres.

- **Ajout de la couche d'Entrée :** On définit la forme des données d'entrée que le modèle attend, sans inclure la taille du lot.

- **Ajout de la couche de mise à plat (Flatten) :** Cette couche transforme les images d'entrée en tableaux 1D pour faciliter leur traitement par les couches suivantes.

- **Ajout des couches cachées Dense :** Premièrement, une couche de 300 neurones avec la fonction d'activation ReLU, puis une seconde couche de 100 neurones, également avec ReLU. Ces couches sont responsables de la computation des transformations des entrées grâce aux poids et aux biais.

- **Ajout de la couche de sortie :** Une couche de 10 neurones, chacun correspondant à une classe, utilisant la fonction d'activation softmax pour prédire la probabilité que l'entrée appartienne à chaque classe.

Chaque étape ajoute des éléments essentiels au modèle pour lui permettre de traiter les données, apprendre des caractéristiques et faire des prédictions.

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.summary()

La fonction Softmax est une extension de la fonction sigmoïde utilisée pour les tâches de classification multiclasse. Voici une explication simplifiée de son fonctionnement :

### Fonction Softmax
Softmax est utilisée principalement dans la couche de sortie des réseaux de neurones pour des problèmes de classification où il y a plus de deux classes. Cette fonction transforme les scores (aussi appelés logits) de chaque classe en probabilités en utilisant l'exponentielle des scores, garantissant que la somme de toutes les probabilités de sortie est égale à 1. La classe avec la plus haute probabilité est généralement choisie comme la classe prédite.

Voici la formule mathématique de Softmax :
$ \text{Softmax}(z_i) = \frac{e^{z_i}}{\sum_{j} e^{z_j}} $
où $ z_i $ est le score de la classe $i$ et le dénominateur est la somme des exponentielles de tous les scores de classe, ce qui normalise ces scores en probabilités.

### Avantages de Softmax
- **Interprétable :** Les sorties peuvent être interprétées directement comme des probabilités de classe.
- **Flexible :** Convient à des problèmes de classification où chaque instance peut appartenir à une et une seule de plusieurs classes.
- **Efficace :** Elle aide à amplifier les différences de probabilité entre les classes, rendant le choix de la classe prédite plus clair.

Softmax est largement utilisée pour la dernière couche des réseaux de neurones dans les tâches de classification multiclasse, aidant le modèle à décider clairement quelle classe est la plus probable pour une entrée donnée.

In [ ]:
# extra code – another way to display the model's architecture
tf.keras.utils.plot_model(model, "my_fashion_mnist_model.png", show_shapes=True)

In [ ]:
model.layers

In [ ]:
hidden1 = model.layers[1]
hidden1.name

In [ ]:
model.get_layer('dense') is hidden1

In [ ]:
weights, biases = hidden1.get_weights()
weights

Remarquez que la couche Dense a initialisé les poids de connexion de manière aléatoire (ce qui est nécessaire pour briser la symétrie, comme nous l'avons vu plus haut), et que les biais ont été initialisés à des zéros, ce qui est correct.

In [ ]:
weights.shape

In [ ]:
biases

In [ ]:
biases.shape

### Compiling the model

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

Nous utilisons la fonction de perte "sparse_categorical_crossentropy" parce que nous avons des étiquettes éparses (c'est-à-dire, pour chaque instance, il y a juste un indice de classe cible, de 0 à 9 dans ce cas), et les classes sont exclusives.

Si, à la place, nous avions une probabilité cible par classe pour chaque instance (comme des vecteurs "one-hot", par exemple, [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.] pour représenter la classe 3), alors nous devrions utiliser la perte "categorical_crossentropy". Si nous faisions une classification binaire ou une classification binaire multilabel, alors nous utiliserions la fonction d'activation "sigmoid" dans la couche de sortie au lieu de la fonction "softmax", et nous utiliserions la perte "binary_crossentropy".

Concernant l'optimiseur, "sgd" signifie que nous allons entraîner le modèle en utilisant la descente de gradient stochastique. Autrement dit, Keras exécutera l'algorithme de rétropropagation décrit précédemment (c'est-à-dire, l'autodifférenciation en mode inverse plus la descente de gradient).

C'est equivalent :

### Entraînement et évaluation du modèle

Dans le contexte de l'entraînement des réseaux de neurones, les **epochs** désignent le nombre de fois que l'algorithme d'apprentissage traite l'ensemble des données. Une epoch comprend toutes les étapes de propagation avant et de rétropropagation à travers le réseau pour toutes les instances de l'ensemble de données. Voici ce qui se passe pendant une epoch :

1. **Propagation Avant :**
   - Chaque instance de l'ensemble de données est passée à travers le réseau.
   - Le réseau fait des prédictions basées sur son état actuel (poids et biais).
   - Les prédictions sont comparées aux objectifs réels à l'aide d'une fonction de perte pour calculer la performance du modèle.

2. **Propagation Arrière (Rétropropagation) :**
   - La perte est utilisée pour calculer le gradient de la fonction de perte par rapport à chaque poids et biais dans le réseau.
   - Ces gradients sont utilisés pour mettre à jour les poids et les biais afin de minimiser la perte. Les mises à jour sont effectuées de manière à idéalement diminuer la perte dans les prédictions ultérieures.

3. **Itération sur les Lots :**
   - Les ensembles de données sont généralement divisés en petits ensembles appelés lots, surtout lorsqu'ils sont trop grands pour tenir en mémoire en une seule fois.
   - Une epoch comprend un ou plusieurs lots, selon la taille de l'ensemble de données et la taille du lot. Chaque lot subit sa propre propagation avant et arrière.
   - Le nombre de lots traités en une epoch est égal au nombre total d'échantillons divisé par la taille du lot.

4. **Répétition :**
   - Ce processus est répété pour un nombre spécifié d'epochs. À chaque epoch, le modèle apprend idéalement des motifs qui reflètent mieux les données et améliore ainsi sa précision et sa performance générale.

**Choix du Nombre d'Époques :**
Le nombre optimal d'epochs est généralement déterminé expérimentalement à travers des techniques telles que la validation croisée et le suivi de la perte de validation pendant l'entraînement. Des techniques comme l'arrêt précoce sont également utilisées, où l'entraînement peut être arrêté automatiquement lorsque la perte de validation commence à augmenter, signalant que le modèle peut commencer à surapprendre.

In [ ]:
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))

In [ ]:
history.params

In [ ]:
print(history.epoch)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

pd.DataFrame(history.history).plot(
    figsize=(8, 5), xlim=[0, 29], ylim=[0, 1], grid=True, xlabel="Epoch",
    style=["r--", "r--.", "b-", "b-*"])
plt.legend(loc="lower left")  # extra code
save_fig("keras_learning_curves_plot")  # extra code
plt.show()

### Using the model to make predictions

In [ ]:
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

In [ ]:
y_pred = y_proba.argmax(axis=-1)
y_pred

In [ ]:
np.array(class_names)[y_pred]

In [ ]:
y_new = y_test[:3]
y_new

In [ ]:
# this cell generates and saves Figure 10–12
plt.figure(figsize=(7.2, 2.4))
for index, image in enumerate(X_new):
    plt.subplot(1, 3, index + 1)
    plt.imshow(image, cmap="binary", interpolation="nearest")
    plt.axis('off')
    plt.title(class_names[y_test[index]])
plt.subplots_adjust(wspace=0.2, hspace=0.5)
save_fig('fashion_mnist_images_plot', tight_layout=False)
plt.show()

## Construction d'un MLP de régression à l'aide de l'API séquentielle

Revenons au problème du logement en Californie et abordons-le en utilisant un perceptron multicouche (MLP) construit avec Keras, en utilisant la même architecture qu'auparavant avec trois couches cachées de 50 neurones chacune. Cependant, cette fois, nous construirons le modèle en utilisant l'API séquentielle de Keras. Le processus de construction, d'entraînement, d'évaluation et d'utilisation d'un MLP de régression est similaire à celui de la classification, avec quelques différences clés :

- **Couche de sortie :** Le modèle aura un seul neurone de sortie car nous ne prévoyons qu'une seule valeur, telle que le prix d'une maison. Ce neurone de sortie n'utilise pas de fonction d'activation, ce qui lui permet de produire une gamme de valeurs continues.

- **Fonction de perte :** Nous utilisons l'erreur quadratique moyenne (MSE) comme fonction de perte. La MSE est standard pour les problèmes de régression et fonctionne en pénalisant le carré de l'erreur entre les prédictions et les valeurs réelles.

- **Métrique :** Nous mesurons la performance en utilisant l'Erreur Quadratique Moyenne Racine (RMSE), qui fournit une métrique d'erreur sensible à l'échelle dans les mêmes unités que la valeur cible, ce qui facilite son interprétation.

- **Optimiseur :** L'optimiseur Adam est utilisé, connu pour son efficacité à gérer les gradients épars et à adapter le taux d'apprentissage pendant la formation, ce qui convient à une large gamme de problèmes.

- **Couche de normalisation :** Au lieu d'une couche Flatten (utilisée dans les tâches de classification pour les données d'image), nous incluons une couche de normalisation comme première couche de notre réseau. Cette couche agit comme le StandardScaler de Scikit-Learn, standardisant les caractéristiques d'entrée pour avoir une moyenne nulle et une variance unitaire. De manière cruciale, la couche de normalisation doit être "adaptée" aux données d'entraînement en utilisant la méthode `adapt()` avant de procéder à l'ajustement du modèle. Cela garantit que la couche échelonne correctement les caractéristiques d'entrée pendant l'entraînement et l'inférence.

In [ ]:
# load and split the California housing dataset

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [ ]:
tf.random.set_seed(42)
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

norm_layer.adapt(X_train)

history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))
mse_test, rmse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

In [ ]:
rmse_test

In [ ]:
y_pred

In [ ]:
y_new = y_test[:3]
y_new